In [1]:
%autosave 60
%load_ext autoreload
%autoreload 2
%matplotlib inline


Autosaving every 60 seconds


In [2]:
import json
import os
import pickle
import sys
from collections import Counter
from io import BytesIO
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union, cast

import cv2
import matplotlib as plt
import numpy as np
import pandas as pd
import PIL.Image as pil_img
import seaborn as sns
import sklearn as skl
from IPython.display import Image, display
from matplotlib.patches import Rectangle
from matplotlib_inline.backend_inline import set_matplotlib_formats
from pycocotools.helpers import CocoClassDistHelper as COCO
from pycocotools.helpers import CocoJsonBuilder, check_boxes, check_stats
from tqdm import tqdm

from geoscreens.consts import VIDEO_PATH
from geoscreens.data.metadata import get_all_metadata, get_geoguessr_split_metadata, load_metadata
from geoscreens.data.splitting import get_images_with_metadata, generate_train_val_splits, get_metadata_df

In [3]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 15)
pd.set_option("display.max_rows", 50)
# Suitable default display for floats
pd.options.display.float_format = "{:,.2f}".format
plt.rcParams["figure.figsize"] = (12, 10)

# This one is optional -- change graphs to SVG only use if you don't have a
# lot of points/lines in your graphs. Can also just use ['retina'] if you
# don't want SVG.
%config InlineBackend.figure_formats = ["retina"]
set_matplotlib_formats("pdf", "png")

In [4]:
coco_path = Path("/home/gbiamby/proj/geoscreens/datasets/geoscreens_012/geoscreens_012.json")
gs = COCO(coco_path)

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
num images: 2218
num annotations: 9474


In [5]:
all_metadata = get_all_metadata()
df_meta = get_metadata_df(all_metadata)

Total video files found:  4084


  0%|          | 0/4084 [00:00<?, ?it/s]

Length of metadata for split='train':  3493
Length of metadata for split='val':  39
Length of metadata for split='test':  100


In [6]:
img_data = get_images_with_metadata(gs, df_meta)

In [7]:
# df_images = pd.DataFrame(img_data)
# df_images.rename(columns={"id": "image_id"}, inplace=True)
# df_images

In [8]:
check_boxes(coco_path)

('invalid x', {'area': 4761, 'bbox': [1053, 686, 207, 23], 'category_id': 32, 'id': 1, 'ignore': 0, 'image_id': 0, 'iscrowd': 0, 'segmentation': []})
('invalid width', {'area': 341700, 'bbox': [0, 375, 1139, 300], 'category_id': 22, 'id': 6, 'ignore': 0, 'image_id': 1, 'iscrowd': 0, 'segmentation': []})
('invalid y', {'area': 8954, 'bbox': [448, 546, 242, 37], 'category_id': 64, 'id': 8, 'ignore': 0, 'image_id': 1, 'iscrowd': 0, 'segmentation': []})
('invalid height', {'area': 87135, 'bbox': [0, 118, 157, 555], 'category_id': 41, 'id': 120, 'ignore': 0, 'image_id': 26, 'iscrowd': 0, 'segmentation': []})
Found 7039 invalid boxes
Counts:  Counter({'invalid x': 4224, 'invalid width': 1838, 'invalid height': 504, 'invalid y': 473})


In [9]:
# check_stats(coco_path)

---

In [10]:
pd.DataFrame(gs.get_cat_counts().values())


,id,name,supercategory,img_count,ann_count
0,1,battle_royale_btw_rounds_green_box,,125.00,125.00
1,2,battle_royale_btw_rounds_red_box,,17.00,17.00
2,3,battle_royale_end_round_box_red,,2.00,2.00
3,4,battle_royale_game_over_dark_box,,22.00,22.00
4,5,battle_royale_knocked_out_end_game_red,,30.00,31.00
...,...,...,...,...,...
72,73,url,,411.00,413.00
73,74,users_bar_white,,0.00,0.00
74,75,video,,571.00,603.00
75,76,view_summ_green_on_blue,,13.00,13.00


# Anchor Boxes

In [17]:
from pycocotools.helpers.anchors import compute_anchors

results = compute_anchors(coco_path, coco_path.parent / "anchors", [5, 60], stop_at_iou=0.90)


loading annotations into memory...
Done (t=0.04s)
creating index...
index created!

kmeans clustering, k =  5
Labels:  [3 3 0 ... 2 3 0]
Clusters:  [[ 93.00422142  34.02458791]
 [274.76523955  46.54896364]
 [157.46240225 181.07219494]
 [565.7138448  199.75778954]
 [219.32905983 502.17004049]]
h/w ratios:  [0.36583918 0.16941358 1.14993924 0.35310748 2.28957367]
Avg. IoU:  0.5933114079280778

kmeans clustering, k =  6
Labels:  [1 1 5 ... 0 1 5]
Clusters:  [[ 92.98551648  33.7133877 ]
 [274.89844791  49.09672262]
 [148.21879169 176.29807509]
 [590.28542304 187.15324499]
 [345.99580336 415.71223022]
 [109.16455026 555.2626455 ]]
h/w ratios:  [0.36256601 0.17859949 1.18944483 0.31705551 1.20149501 5.08647399]
Avg. IoU:  0.6212877640588035

kmeans clustering, k =  7
Labels:  [5 5 4 ... 3 0 4]
Clusters:  [[ 79.10209619  33.85781893]
 [183.4329729   34.02398281]
 [322.46697154  57.52845528]
 [149.3037256  175.65015015]
 [599.43727005 197.55114054]
 [345.99580336 415.71223022]
 [109.16455026 5

In [30]:
nc = 20
print(results[nc]["iou"])
df = pd.DataFrame(results[nc]["clusters"], columns=["width", "height"])
df["ratio"] = df.height / df.width
df["base_size"] = df.height / 8.0
display(df)
print(",".join([str(bs) for bs in [4,5,6,8,15,26,31,48,65,69]]))
print(",".join([f"{bs:0.02f}" for bs in df.ratio.values]))

print(",".join([str(bs) for bs in [4,6,23,52,63]]))
print(",".join([str(bs) for bs in [3,4,5,22,52,64]]))
print(",".join([f"{bs:0.02f}" for bs in df.ratio.values]))

0.7932941527713548


,width,height,ratio,base_size
0,410.47,25.33,0.06,3.17
1,61.02,30.11,0.49,3.76
2,95.40,30.83,0.32,3.85
3,286.01,32.58,0.11,4.07
4,158.84,32.72,0.21,4.09
5,215.05,36.05,0.17,4.51
6,545.04,45.31,0.08,5.66
7,277.15,81.72,0.29,10.22
8,365.79,93.82,0.26,11.73
9,97.69,114.78,1.17,14.35


4,5,6,8,15,26,31,48,65,69
0.06,0.49,0.32,0.11,0.21,0.17,0.08,0.29,0.26,1.17,1.25,0.32,0.78,0.57,1.95,0.45,1.31,3.54,1.25,5.60
4,6,23,52,63
3,4,5,22,52,64
0.06,0.49,0.32,0.11,0.21,0.17,0.08,0.29,0.26,1.17,1.25,0.32,0.78,0.57,1.95,0.45,1.31,3.54,1.25,5.60


In [2]:
print(",".join([str(bs) for bs in [4,5,6,8,15,26,31,48,65,69]]))
print(",".join([str(8*bs) for bs in [4,5,6,8,15,26,31,48,65,69]]))

4,5,6,8,15,26,31,48,65,69
32,40,48,64,120,208,248,384,520,552


In [45]:
ratios = [0.03232666, 0.32466155, 0.16829857, 0.06960362, 0.13733681, 0.26874476
, 0.21486479, 0.12025294, 0.10558306, 0.0686041, 0.05882353, 0.095259
, 0.10972197, 0.03576037, 0.05837962, 0.05233987, 0.16004019, 0.15555059
, 0.72779865, 0.56002639, 0.67524641, 0.74870974, 0.6576581,  0.41884503
, 0.57760766, 0.17744202, 0.7537875, 0.39796285, 0.25761093, 0.2220346
, 0.76633415, 0.36426718, 0.26349288, 0.52401231, 1.35222693, 0.27887195
, 0.71730647, 0.52501349, 0.76417979, 1.00861367, 2.57618869, 0.65038579
, 1.00513278, 1.82245827, 0.73063804, 2.34746611, 3.2583827, 3.12290856
, 3.47869761, 0.7429066 ]

sorted(ratios)


50

In [46]:
dims = [[836.08333333, 27.02777778],
[94.10889292, 30.55353902],
[202.06970509, 34.0080429, ],
[494.24817518, 34.40145985],
[253.79518072, 34.85542169],
[130.28324946, 35.01294033],
[166.73361082, 35.8251821, ],
[301.01072961, 36.19742489],
[347.28537736, 36.66745283],
[572.12323944, 39.25, ],
[685.93023256, 40.34883721],
[440.12784091, 41.92613636],
[393.22346369, 43.1452514, ],
[1276.47058824, 45.64705882],
[963.40540541, 56.24324324],
[1092.05263158, 57.15789474],
[554.37025316, 88.72151899],
[680.89130435, 105.91304348],
[166.62612613, 121.27027027],
[239.36842105, 134.05263158],
[204.12301587, 137.83333333],
[247.88268156, 185.59217877],
[289.76056338, 190.56338028],
[478.18852459, 200.28688525],
[360.89830508, 208.45762712],
[1278.13978495, 226.79569892],
[303.87195122, 229.05487805],
[578.85714286, 230.36363636],
[936.975, 241.375, ],
[1266.69444444, 281.25, ],
[378.425, 290., ],
[797.65116279, 290.55813953],
[1136.76470588, 299.52941176],
[592.925, 310.7, ],
[238.8375, 322.9625, ],
[1273.18867925, 355.05660377],
[552.18309859, 396.08450704],
[794.42857143, 417.08571429],
[597.62068966, 456.68965517],
[464.37837838, 468.37837838],
[200.325, 516.075, ],
[832.27710843, 541.30120482],
[560.125, 563., ],
[309.53030303, 564.10606061],
[836.28421053, 611.02105263],
[262.75438596, 616.80701754],
[189.66292135, 617.99438202],
[204.75925926, 639.44444444],
[199.86923077, 695.28461538],
[949.07142857, 705.07142857],]

In [54]:
def scale_dim(w, h, from_w=1280, from_h=720, dim_to=640):
    return (
        w * (dim_to / from_w),
        h * (dim_to / from_h),
        (h * (dim_to / from_h)) / (w * (dim_to / from_w)),
    )

# [
#     tf_dim(dim[0], dim[1])
#     for dim in [
#         [1280, 720],
#         [1280/2, 720],
#         [1280/4, 720],
#         [1280, 720/2],
#         [1280, 720/4],
#     ]
# ]
centroids_scaled = [tf_dim(dim[0], dim[1]) for dim in dims]
sorted(centroids_scaled, key=lambda x: x[2])
[0.05,0.11,0.16,0.19,0.27,0.39,0.]


[(418.041666665, 24.02469136, 0.05746960955270595),
 (638.23529412, 40.57516339555555, 0.06357398873012915),
 (546.02631579, 50.807017546666664, 0.09304866098469672),
 (481.702702705, 49.993993991111104, 0.10378599437032425),
 (342.96511628, 35.86563307555555, 0.10457516340021736),
 (286.06161972, 34.888888888888886, 0.12196284465926775),
 (247.12408759, 30.57907542222222, 0.12373976054068644),
 (220.063920455, 37.267676764444445, 0.1693493267201207),
 (173.64268868, 32.59329140444444, 0.18770321775257393),
 (196.611731845, 38.35133457777778, 0.1950612723762195),
 (150.505364805, 32.17548879111111, 0.2137830025713621),
 (126.89759036, 30.982597057777777, 0.2441543371302971),
 (340.445652175, 94.14492753777778, 0.27653438055770574),
 (277.18512658, 78.86357243555555, 0.28451588802256417),
 (101.034852545, 30.229371466666663, 0.2991974621153901),
 (639.069892475, 201.59617681777777, 0.3154524711483949),
 (83.36680541, 31.844606311111107, 0.3819818470253064),
 (633.34722222, 250.0, 0.3947

In [27]:
from mmdet.core.anchor import AnchorGenerator

ag = AnchorGenerator(
    strides=[2],
    scales=[4,8],
    ratios=[1.0],
    base_sizes=[2],
)
print(ag.grid_priors([(2, 2)], device="cpu"))

# base size gets multiplied by scales, and that gives the anchor box sizes
# strides don't get multiplied

# print([[s] for s in [4, 8, 16, 32, 64]])
ag = AnchorGenerator(
    strides=[4, 8, 16, 32, 64],
    # ratios=[0.08, 0.16, 0.25, 0.36, 0.5, 0.7, 1, 2],
    ratios=[1.0],
    scales=[8],
)

ag.grid_priors([(2, 2)])

[tensor([[-4., -4.,  4.,  4.],
        [-8., -8.,  8.,  8.],
        [-2., -4.,  6.,  4.],
        [-6., -8., 10.,  8.],
        [-4., -2.,  4.,  6.],
        [-8., -6.,  8., 10.],
        [-2., -2.,  6.,  6.],
        [-6., -6., 10., 10.]])]


AssertionError: 